<a href="https://colab.research.google.com/github/bhaveshsingh0206/Machine-Translator/blob/master/Simple_Machine_Translartor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [0]:
MAX_VOCAB = 20000
DIMENSIONS = 300
LSTM_UNITS = 256
NUM_SAMPLES = 20000 
BATCH_SIZE = 128  
EPOCHS = 30

In [0]:
t = 0
input_sequences = []
target_input_sequences = []
target_target_sequences = []
with open('fra.txt') as f:
  for i,line in enumerate(f):
    t+=1
    if t>NUM_SAMPLES:
      break
    line = line.rstrip().split('\t')
    input_text = line[0]
    target_input = '<sos> ' + line[1]
    temp = line[1] + ' <eos>'
    input_sequences.append(input_text)
    target_input_sequences.append(target_input)
    target_target_sequences.append(temp)

In [0]:
# Loading file from Drive
import pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# downloaded = drive.CreateFile({'id':"1CJCcOIF5-6ceTotmjnF2Lm7JgEMm4zvJ"}) 
# downloaded.GetContentFile('glove.6B.300d.txt')  

In [0]:
word2Vec = {}
with open('glove.6B.300d.txt') as f:
  for line in f:
    line = line.split(' ')
    word = line[0]
    word2Vec[word] = line[1:]

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Input
from keras.layers import Dense, LSTM, Embedding, RepeatVector

Using TensorFlow backend.


In [0]:
tokenizer_inputs = Tokenizer(num_words=MAX_VOCAB)
tokenizer_inputs.fit_on_texts(input_sequences)
input_sequences = tokenizer_inputs.texts_to_sequences(input_sequences)

In [0]:
total = target_input_sequences+target_target_sequences
tokenizer_outputs = Tokenizer(num_words=MAX_VOCAB)
tokenizer_outputs.fit_on_texts(total)
target_input_sequences = tokenizer_outputs.texts_to_sequences(target_input_sequences)
target_target_sequences = tokenizer_outputs.texts_to_sequences(target_target_sequences)

In [0]:
input_vocab_list = tokenizer_inputs.word_index
output_vocab_list = tokenizer_outputs.word_index

In [0]:
num_words_input = min(MAX_VOCAB, len(input_vocab_list)+1)
num_words_output = min(MAX_VOCAB, len(output_vocab_list)+1)

In [0]:
max_input_len = max(len(s) for s in input_sequences)
max_target_len = max(len(j) for j in target_target_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_input_len)

In [0]:
target_input_sequences = pad_sequences(target_input_sequences, maxlen=max_target_len, padding='post')

In [0]:
target_target_sequences = pad_sequences(target_target_sequences, maxlen=max_target_len, padding='post')

In [0]:
embedding_matrix = np.zeros((num_words_input, DIMENSIONS))
for word, i in input_vocab_list.items():
  embedding_vector = word2Vec.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [0]:
# Now its time to create rocking stuff

In [0]:
encoder_embedding = Embedding(num_words_input, DIMENSIONS, trainable = True, weights=[embedding_matrix])

In [0]:
encoder_input = Input(shape=(max_input_len,))

In [0]:
x = encoder_embedding(encoder_input)

In [0]:
lstm = LSTM(LSTM_UNITS, return_state=True)

In [0]:
encoder_outputs, h, c = lstm(x)

In [0]:
decoder_input = Input(shape=(max_target_len,))
decoder_embedding = Embedding(num_words_output, DIMENSIONS)
decoder_x = decoder_embedding(decoder_input)

In [0]:
decoder_initial_state = [h,c]
decoder_lstm = LSTM(LSTM_UNITS, return_sequences=True, return_state=True)
decoder_output, _, _ = decoder_lstm(decoder_x, initial_state=decoder_initial_state)

In [0]:
decoder_dense = Dense(num_words_output, activation='softmax')
decoder_final_output = decoder_dense(decoder_output)

In [0]:
targets_one_hot = np.zeros((len(target_input_sequences), max_target_len, num_words_output))

for i, d in enumerate(target_target_sequences):
  for t, word in enumerate(d):
    if word > 0:
      targets_one_hot[i, t, word] = 1

In [0]:
model = Model([encoder_input, decoder_input], decoder_final_output)

In [0]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 6)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 13)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 6, 300)       1054500     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 13, 300)      2166000     input_2[0][0]                    
____________________________________________________________________________________________

In [0]:
model.fit([input_sequences, target_input_sequences], targets_one_hot, epochs=EPOCHS, batch_size=BATCH_SIZE)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/30
20000/20000 [==============================] - 99s 5ms/step - loss: 2.0187 - accuracy: 0.0906
Epoch 2/30
20000/20000 [==============================] - 98s 5ms/step - loss: 1.6636 - accuracy: 0.1106
Epoch 3/30
20000/20000 [==============================] - 98s 5ms/step - loss: 1.4415 - accuracy: 0.1370
Epoch 4/30
20000/20000 [==============================] - 97s 5ms/step - loss: 1.2657 - accuracy: 0.1607
Epoch 5/30
20000/20000 [==============================] - 97s 5ms/step - loss: 1.1254 - accuracy: 0.1764
Epoch 6/30
20000/20000 [==============================] - 97s 5ms/step - loss: 1.0098 - accuracy: 0.1888
Epoch 7/30
20000/20000 [==============================] - 97s 5ms/step - loss: 0.9102 - accuracy: 0.1987
Epoch 8/30
20000/20000 [==============================] - 100s 5ms/step - loss: 0.8241 - accuracy: 0.2082
Epoch 9/30
20000/20000 [==============================] - 97s 5ms/step - loss: 0.7468 - accuracy: 0.2169
Epoch 10/30
20000/20000 [=============================

In [0]:
# model.save('translator.h5')

In [0]:
# from google.colab import files
# files.download('translator.h5')

In [0]:
import keras
model = keras.models.load_model('translator.h5')

In [0]:
encoding_model = Model(encoder_input, decoder_initial_state)
decoder_inputs_h = Input(shape=(LSTM_UNITS,))
decoder_inputs_c = Input(shape=(LSTM_UNITS,))
decorder_initial_states = [decoder_inputs_h, decoder_inputs_c]

In [0]:
inpu = Input(shape=(1,))
decorder_embedded = decoder_embedding(inpu)
decoder_predict_output, hi, ci = decoder_lstm(decorder_embedded, initial_state=decorder_initial_states)

In [0]:
decoder_states = [hi, ci]

In [0]:
decoder_outputs = decoder_dense(decoder_predict_output)

In [0]:
decoder_model = Model([inpu]+decorder_initial_states, [decoder_outputs]+ decoder_states)

In [0]:
idx2word_eng = {v:k for k, v in input_vocab_list.items()}
idx2word_trans = {v:k for k, v in output_vocab_list.items()}

In [0]:
def decode_sequence(input_seq):
  # Encode the input as state vectors.
  states_value = encoding_model.predict(input_seq)

  # Generate empty target sequence of length 1.
  target_seq = np.zeros((1, 1))

  # Populate the first character of target sequence with the start character.
  # NOTE: tokenizer lower-cases all words
  target_seq[0, 0] = output_vocab_list['sos']

  # if we get this we break
  eos = output_vocab_list['eos']

  # Create the translation
  output_sentence = []
  for _ in range(max_target_len):
    output_tokens, h, c = decoder_model.predict(
      [target_seq] + states_value
    )
    # output_tokens, h = decoder_model.predict(
    #     [target_seq] + states_value
    # ) # gru

    # Get next word
    idx = np.argmax(output_tokens[0, 0, :])

    # End sentence of EOS
    if eos == idx:
      break

    word = ''
    if idx > 0:
      word = idx2word_trans[idx]
      output_sentence.append(word)

    # Update the decoder input
    # which is just the word just generated
    target_seq[0, 0] = idx

    # Update states
    states_value = [h, c]
    # states_value = [h] # gru

  return ' '.join(output_sentence)



while True:
  # Do some test translations
  input_text = ["free service"]
  input_text = tokenizer_inputs.texts_to_sequences(input_text)
  input_text = pad_sequences(input_text, max_input_len)

  # i = np.random.choice(len(input_texts))
  # input_seq = encoder_inputs[i:i+1]
  translation = decode_sequence(input_text)
  print('-')
  print('Input:', input_text)
  print('Translation:', translation)
  break

-
Input: [[  0   0   0   0   0 240]]
Translation: c'est libre
